<a href="https://colab.research.google.com/github/carlaperez9/ReferenceClassForecasting/blob/main/ININ6095_ReferenceClassForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create dummy data.

In [ ]:
import numpy as np

np.random.seed(0)
# identically sized periods
M = 100
# this is just to generate dummy data for y_m
y_m = np.random.randint(1, 3, size=M)
# number of contiguos and mutually exclusive classes
K = 4
# number of lags
L = 3

Creating a training matrix with $M-L$ rows and $L+1$ columns.

In [ ]:
print("M - L rows = ", M-L)

M - L rows =  97


In [ ]:
print("L+1 columns = ", L+1)

L+1 columns =  4


In [ ]:
# training matrix
T = np.zeros((M-L, L+1))
# predictions are a zero-matrix with dim = M
y_m_hat = np.zeros(M)

In [ ]:
T

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],


In [ ]:
y_m_hat

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
for t in range(L, M):
    # Assign each observation to one of the K classes
    class_assignment = np.random.randint(1, K + 1)
    y_m_hat[t] = class_assignment
    # Construct the training matrix
    T[t - L, :] = [y_m_hat[t - l] for l in range(L, -1, -1)]

Running a simple validation to confirm the shape of the training matrix.

In [ ]:
print(f"The shape of the training matrix is: {T.shape}")

The shape of the training matrix is: (97, 4)


Building $L$ matrices

In [ ]:
transition_matrices = []
# range(1, L+1) ensures we build L matrices
for lag in range(1, L + 1):

    # transition matrix of dim K x K, initialized with dummy data
    transition_matrix = np.zeros((K, K), dtype=int)

    # Loop through the training matrix to tally class transitions
    for i in range(len(T) - lag):
        current_class = int(T[i, lag - 1]) - 1
        next_class = int(T[i, lag]) - 1
        transition_matrix[current_class, next_class] += 1

    transition_matrices.append(transition_matrix)

for lag, transition_matrix in enumerate(transition_matrices):
    print(f"Transition Matrix for lag {lag + 1}:\n{transition_matrix}\n")


Transition Matrix for lag 1:
[[ 4  4  7  7]
 [ 3  5  6  3]
 [ 6  4  6 12]
 [ 9  4  9  7]]

Transition Matrix for lag 2:
[[ 4  4  7  7]
 [ 3  5  6  3]
 [ 6  4  6 12]
 [ 9  4  9  6]]

Transition Matrix for lag 3:
[[ 4  4  7  7]
 [ 3  5  6  3]
 [ 6  4  6 12]
 [ 9  4  9  5]]



Now, each of the $L$ square matrices need to be modified. This is to compute the class transition frequency, defined as $$ p^{l}_{ij} = \frac{f^{l}_{ij}}{∑_{\forall j} f^{l}_{ij}}$$

In [ ]:
class_transition_frequency_matrices = []

for matrix in transition_matrices:
  row_sums = transition_matrix.sum(axis=1, keepdims=True)
  # preliminary - to ensure sums we do not get NaNs
  row_sums[row_sums==0]=1
  class_transition_frequency_matrix = transition_matrix / row_sums
  class_transition_frequency_matrices.append(class_transition_frequency_matrix)

for lag, class_transition_ in enumerate(class_transition_frequency_matrices):
  print(f"Class transition frequency matrix for lag {lag + 1}:\n{class_transition_}\n")


Class transition frequency matrix for lag 1:
[[0.18181818 0.18181818 0.31818182 0.31818182]
 [0.17647059 0.29411765 0.35294118 0.17647059]
 [0.21428571 0.14285714 0.21428571 0.42857143]
 [0.33333333 0.14814815 0.33333333 0.18518519]]

Class transition frequency matrix for lag 2:
[[0.18181818 0.18181818 0.31818182 0.31818182]
 [0.17647059 0.29411765 0.35294118 0.17647059]
 [0.21428571 0.14285714 0.21428571 0.42857143]
 [0.33333333 0.14814815 0.33333333 0.18518519]]

Class transition frequency matrix for lag 3:
[[0.18181818 0.18181818 0.31818182 0.31818182]
 [0.17647059 0.29411765 0.35294118 0.17647059]
 [0.21428571 0.14285714 0.21428571 0.42857143]
 [0.33333333 0.14814815 0.33333333 0.18518519]]



Producing a foreacast:

To produce a forecast for a particular time period $t$ using the clases in the last $L$ previous periods $\widehat{y}_{t-1}, \widehat{y}_{t-2},...,\widehat{y}_{t-L}$, the analyst first presents the following indices for each class $k$, $D^{k}_{t}$.

Note that $$D_k = \sum_{\forall l} p^{l}_{ \widehat {y}_ {t - (L+1-l)^2}}$$

In [ ]:
# this function will calculate d_k for one transition matrix
def D_k(class_transition_frequency_matrix):
  # initializing an empty list to store all values of D_k
  d_k = []
  # abstracting to obtain the length of any class transition frequency matrix
  L = len(class_transition_frequency_matrix)
  # looping over all lags
  for lag in range(1, L+1):
    sub_index = t - (L + 1 - lag)**2
    D_k = np.sum(class_transition_frequency_matrix[sub_index])
    d_k.append(D_k)

  print(f"lag={lag}, sub_index={sub_index}, class_transition_frequency_matrix.shape={class_transition_frequency_matrix.shape}")

  return D_k

In [ ]:
for transition_frequency_matrix in class_transition_frequency_matrices:
  D_k(transition_frequency_matrix)

IndexError: ignored